In [1]:
import riiideducation

In [2]:
import sys
import numpy as np
import pandas as pd
import dask.dataframe as dd
import math
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objs as go
import time
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from sklearn.decomposition import PCA
from tqdm import tqdm
from sklearn.preprocessing import MultiLabelBinarizer
import collections
import optuna
from optuna.samplers import TPESampler

import feather
import os
import gc
import glob
import time

import lightgbm as lgb
from sklearn.linear_model import LogisticRegression

from sklearn.isotonic import IsotonicRegression
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score,precision_recall_curve
from sklearn.metrics import make_scorer,f1_score, recall_score, accuracy_score, average_precision_score, auc
from sklearn.metrics import f1_score, mean_squared_error, r2_score, classification_report, confusion_matrix, accuracy_score, precision_score

import matplotlib.pyplot as plt
import plotly.express as px
%matplotlib inline
import matplotlib.style as style
style.use('fivethirtyeight')
import seaborn as sns
import pickle

import copy
import joblib

In [3]:
N_ROWS = 99271101
N_ROWS_train = 50004223
N_ROWS_test = 2500267
N_ROWS_user_stats = 393656
N_ROWS_content_stats = 13523
N_ROWS_previous_stats_questions = 86867031
def get_index_np(type_data):
    if type_data == 'train':
        final = np.arange(N_ROWS_train)
    if type_data == 'test':
        final = np.arange(N_ROWS_test)
    if type_data == 'user_stats':
        final = np.arange(N_ROWS_user_stats)
    if type_data == 'content_stats':
        final = np.arange(N_ROWS_content_stats)
    if type_data == 'previous_stats_questions':
        final = np.arange(N_ROWS_previous_stats_questions)
    if type_data == 'full_dataset':
        final = np.arange(N_ROWS)
    return final

In [4]:
def load_data_np(file_path, columns_selected, type_data):
    data = np.load(file_path , allow_pickle=True)
    df = pd.DataFrame(index=get_index_np(type_data))
    for col in tqdm(columns_selected):
        df[col] = data[col]
    return df

In [5]:
# Load Model

# save model
#joblib.dump(my_model, 'lgb.pkl')
# load model
model = joblib.load('/kaggle/input/runrunrun/model_final_with_pa_filled_with_mean_and_group_no_real_count.pkl')

In [6]:
%%time
#Check features data
file_path = '/kaggle/input/model-features/users_stats_final_to_predict.npz'
columns = ['user_id',
           'user_answered_correctly_mean']


train_answers_by_user_id = load_data_np(file_path, columns, 'user_stats')


100%|██████████| 2/2 [00:00<00:00, 15.51it/s]

CPU times: user 115 ms, sys: 13.1 ms, total: 128 ms
Wall time: 188 ms


In [7]:
%%time
#Check features data
file_path = '/kaggle/input/model-features/content_stats_final_to_predict.npz'
columns = ['content_id',
           'content_answered_correctly_mean']

train_answers_by_content_id = load_data_np(file_path, columns, 'content_stats')

100%|██████████| 2/2 [00:00<00:00, 397.66it/s]

CPU times: user 9.42 ms, sys: 1.06 ms, total: 10.5 ms
Wall time: 24.2 ms


In [8]:
def get_state(train_answers_by_user_id):
    # create DataFrame of features
    file_path = '/kaggle/input/runrunrun/content_ids_to_predict.npz'
    columns = [ 'user_id',
                'content_ids',
                'count_question_anwsered',
                'answered_correctly_user',
                'group',
                'group2']
    data = load_data_np(file_path, columns, 'user_stats')
    #data['group'] = data['group'].apply(np.uint64)
    #data['group2'] = data['group2'].apply(np.uint64)
    #data['count_question_anwsered'] = data['count_question_anwsered'].apply(np.uint64)
    #data['answered_correctly_user'] = data['answered_correctly_user'].apply(np.uint64)
    #data['user_id'] = data['user_id'].apply(np.uint64)
    print('data loaded')  
    
    
    # compute user features over all train data
    user_answered_correctly_mean = train_answers_by_user_id[['user_id', 'user_answered_correctly_mean']]
    user_answered_correctly_mean.set_index('user_id', inplace=True, drop=True)
    user_answered_correctly_mean = user_answered_correctly_mean.values.reshape(user_answered_correctly_mean.shape[0],)
    print('user_answered_correctly_mean done')
    
    # fill dictionary with default values
    state = dict()
    for user_id in data['user_id'].unique():
        state[user_id] = {}
    total = len(state.keys())
    print('total state done')
    del [[train_answers_by_user_id]]
    gc.collect()
    train_answers_by_user_id = pd.DataFrame()
    # add user content attempts
    user_content = data['content_ids']
    user_answered_correctly_count = data['count_question_anwsered']
    answered_correctly_user = data['answered_correctly_user']
    group = data['group']
    group2 = data['group2']
    del [[data]]
    gc.collect()
    data = pd.DataFrame()
    print('user_content done')
    file_path = '/kaggle/input/model-features/previous_arrays_to_predict.npz'
    columns = ['pf_list', 'pc_list']
    pfc = load_data_np(file_path, columns, 'user_stats')
    #pfc = np.load('/kaggle/input/model-features/previous_arrays_to_predict.npz' , allow_pickle=True)
    previous_false_answer = pfc['pf_list'].apply(np.uint8)
    print('previous_false_answer done')
    previous_correct_answer =  pfc['pc_list'].apply(np.uint8)
    print('previous_correct_answer done')
    del [[pfc]]
    gc.collect()
    pfc = pd.DataFrame()
#    for user_id, content,pfa, pca in tqdm(zip(state.keys(), user_content, previous_false_answer, previous_correct_answer),total=total):
#        state[user_id]['previous_correct_answer'] = dict(zip(content, zip(pca, pfa)))
    for user_id, content, pca in tqdm(zip(state.keys(), user_content, previous_false_answer),total=total):
        state[user_id]['previous_false_answer'] = dict(zip(content, pca))
        
    del [[user_content, previous_correct_answer, previous_false_answer]]
    gc.collect()
    user_content = pd.DataFrame()
    previous_correct_answer = pd.DataFrame()
    previous_false_answer = pd.DataFrame()
    for idx, user_id in enumerate(state.keys()):
        state[user_id]['user_answered_correctly_mean'] = user_answered_correctly_mean[idx]
        state[user_id]['user_answered_correctly_count'] = user_answered_correctly_count.values[idx]
        state[user_id]['answered_correctly_user'] = answered_correctly_user.values[idx]
        state[user_id]['group'] = group.values[idx]
        state[user_id]['group2'] = group2.values[idx]
    return state

state = get_state(train_answers_by_user_id)
print('Example of the state for user 2746, attempt counting starts at 0 as the pandas cumcount function is used to create the attempt feature')
display(state[2746])

100%|██████████| 6/6 [00:12<00:00,  2.13s/it]


data loaded
user_answered_correctly_mean done


  0%|          | 0/2 [00:00<?, ?it/s]

total state done
user_content done


100%|██████████| 2/2 [00:06<00:00,  3.28s/it]


previous_false_answer done
previous_correct_answer done


100%|██████████| 393656/393656 [00:35<00:00, 11059.58it/s]


Example of the state for user 2746, attempt counting starts at 0 as the pandas cumcount function is used to create the attempt feature


{'previous_false_answer': {236: 0,
  294: 0,
  297: 1,
  382: 0,
  404: 0,
  405: 0,
  484: 0,
  531: 0,
  714: 0,
  758: 0,
  775: 1,
  873: 0,
  1014: 0,
  1295: 0,
  5273: 0,
  5976: 0,
  10684: 0},
 'user_answered_correctly_mean': 0.5789473684210527,
 'user_answered_correctly_count': 19,
 'answered_correctly_user': 11,
 'group': 4939,
 'group2': 2}

In [9]:
# GET dict 
dict_test = np.load('/kaggle/input/runrunrun/dict_test_invert.npy', allow_pickle=True
)
dict_test = dict_test.item()
#val_list = list(dict_test.values())
#key_list = list(dict_test.keys())
dict_test_invert = {v: k for k, v in dict_test.items()}

dict_test2 = np.load('/kaggle/input/runrunrun/dict_test_invert2.npy', allow_pickle=True
)
dict_test2 = dict_test2.item()
#val_list2 = list(dict_test2.values())
#key_list2 = list(dict_test2.keys())
dict_test2_invert = {v: k for k, v in dict_test2.items()}

In [10]:
# Mapping_question
mapping_question = np.load('/kaggle/input/runrunrun/mapping_question.npy', allow_pickle=True
)
mapping_question = mapping_question.item()
mapping_question2 = np.load('/kaggle/input/runrunrun/mapping_question2.npy', allow_pickle=True
)
mapping_question2 = mapping_question2.item()

In [11]:
def get_user_data(state, test_df):
    # updated data
    user_answered_correctly_mean,\
    answered_correctly_user,\
    previous_false_answer,\
    user_answered_correctly_count,\
    group,\
    group2\
    = [], [], [], [], [], []
    
    for idx, (user_id, content_id) in test_df[['user_id', 'content_id']].iterrows():
        # check if user exists
        if user_id in state:
            # check if user already answered the question, if so update it to a maximum of 4
            if content_id in state[user_id]['previous_false_answer']:
                print('nothing to do')
            else:
                state[user_id]['previous_false_answer'][content_id] = 0
                
            print('user id {}'.format(user_id))
            print('content id {}'.format(content_id))
            if content_id in mapping_question:
                i = mapping_question.get(content_id)
                group_string = dict_test_invert.get(state[user_id]['group'])
                print('group_string group1= {}'.format(group_string))
                group_string = group_string[:i] + '1' + group_string[i+1:]
                print('group_string group1 updated= {}'.format(group_string))
                if group_string in dict_test:
                    value = dict_test.get(group_string)
                else:
                    value = len(dict_test) + 1
                    dict_test[group_string] = value
                    dict_test_invert[value] = group_string
                    print('old value group1 = {}'.format(state[user_id]['group']))
                    print('new value group1 updated= {}'.format(value))
                state[user_id]['group'] = value

            if content_id in mapping_question2:
                i = mapping_question2.get(content_id)
                group_string = dict_test2_invert.get(state[user_id]['group2'])
                print('group_string group2= {}'.format(group_string))
                group_string = group_string[:i] + '1' + group_string[i+1:]
                print('group_string group2 updated= {}'.format(group_string))
                if group_string in dict_test2:
                    value = dict_test2.get(group_string)
                else:
                    value = len(dict_test2) + 1
                    dict_test2[group_string] = value
                    dict_test2_invert[value] = group_string
                print('old value group2 = {}'.format(state[user_id]['group2']))
                print('new value group2 updated= {}'.format(value))
                state[user_id]['group2'] = value
            
        else:
            dict_keys = ['user_answered_correctly_mean',
                         'answered_correctly_user',
                         'user_answered_correctly_count',
                         'previous_false_answer',
                         'group',
                         'group2']
            dict_default_vals = [0.680, 0, 0, dict(zip([content_id],[0])), 9, 2] # Check group 0 string
            state[user_id] = dict(zip(dict_keys, dict_default_vals))
            
            
        # add user data to lists

        user_answered_correctly_mean.append(state[user_id]['user_answered_correctly_mean'])
        answered_correctly_user.append(state[user_id]['answered_correctly_user'])
        previous_false_answer.append(state[user_id]['previous_false_answer'][content_id])
        user_answered_correctly_count.append(state[user_id]['user_answered_correctly_count'])
        group.append(state[user_id]['group'])
        group2.append(state[user_id]['group2'])
    
    return  user_answered_correctly_mean,\
            answered_correctly_user,\
            previous_false_answer,\
            user_answered_correctly_count,\
            group,\
            group2

In [12]:
# updates the user data
def update_user_data(state, prev_test_df):
    for user_id, content_id, answered_correctly, p_timestamp in prev_test_df[['user_id', 'content_id', 'answered_correctly', 'timestamp']].values:
        #print(type(group))
        #print(group)
        #print(type(group2))
        #print(group2)
        #print(type(content_id))
        #print(content_id)
        # update user features
        state[user_id]['answered_correctly_user'] += answered_correctly
        state[user_id]['user_answered_correctly_count'] += 1
        state[user_id]['user_answered_correctly_mean'] = state[user_id]['answered_correctly_user'] / state[user_id]['user_answered_correctly_count']
        if answered_correctly == 1:
            answered_incorrectly = 0
        elif answered_correctly == 0:
            answered_incorrectly = 1
        state[user_id]['previous_false_answer'][content_id] += answered_incorrectly



In [13]:
env = riiideducation.make_env()
iter_test = env.iter_test()

In [14]:
#Define Features to predict
features = [
         'group',
         'group2',
         'content_id',
         'prior_question_elapsed_time',
         'part',
         'previous_false_answer',
         'content_answered_correctly_mean',
         'user_answered_correctly_mean',
         'user_answered_correctly_count',
         'hmean_user_content_accuracy',
         'answered_correctly_user']
#target = features
#target.append('user_id')
#target

In [15]:
%%time
prev_test_df = None

for idx, (test_df, _) in tqdm(enumerate(iter_test)):
    # from 2nd iteration, update user data
    if prev_test_df is not None:
        prev_test_df['answered_correctly'] = eval(test_df['prior_group_answers_correct'].iloc[0])
        update_user_data(state, prev_test_df.loc[prev_test_df['content_type_id'] == 0])
        if idx is 1:
            display(test_df)
            display(prev_test_df)
    
    # get user data from state and update attempt
    user_answered_correctly_mean,\
    answered_correctly_user,\
    previous_false_answer,\
    user_answered_correctly_count,\
    group,\
    group2 = get_user_data(state, test_df)

    # set updated user data
    test_df['previous_false_answer'] = previous_false_answer
    test_df['user_answered_correctly_mean'] = user_answered_correctly_mean
    test_df['answered_correctly_user'] = answered_correctly_user
    test_df['user_answered_correctly_count'] = user_answered_correctly_count
    test_df['group'] = group
    test_df['group2'] = group2
    print(test_df['user_id'])
    print('content')
    print(test_df['content_id'])
    print(test_df['group2'])
    
    
    # Add questions to train
    questions = pd.read_csv('../input/riiid-test-answer-prediction/questions.csv')
    questions.rename(columns={'question_id':'content_id'}, inplace=True)
    test_df = test_df.merge(questions, how = 'left', on = 'content_id')
    
    # Add content stats
    test_df = test_df.merge(train_answers_by_content_id, how = 'left', on = 'content_id')
    
    
    # fill prior question had explenation
    test_df['prior_question_elapsed_time'].fillna(23916, inplace=True)
    
    
    
    # Calculate metrics hmean
    test_df['hmean_user_content_accuracy'] = 2 * (
    (test_df['user_answered_correctly_mean'] * test_df['content_answered_correctly_mean']) /
    (test_df['user_answered_correctly_mean'] + test_df['content_answered_correctly_mean']))

    test_df['answered_correctly'] = model.predict(test_df[features])

    env.predict(test_df.loc[test_df['content_type_id'] == 0, ['row_id', 'answered_correctly']])

    # set previour test_df
    prev_test_df = test_df.copy()

0it [00:00, ?it/s]

nothing to do
user id 554169193
content id 12010
user id 1720860329
content id 457
user id 288641214
content id 13262
user id 1728340777
content id 6119
group_string group1= 0000000000000000000000000000000010000100000000100000100000010001010100000
group_string group1 updated= 0000000000000000000000000000000010000100000000100000100000010001010110000
old value group1 = 779323
new value group1 updated= 1955275
nothing to do
user id 1364159702
content id 12023
user id 1521618396
content id 574
user id 1317245193
content id 12043
user id 1700555100
content id 7910
user id 998511398
content id 7908
user id 1422853669
content id 1143
user id 1096784725
content id 11033
user id 1096784725
content id 11032
user id 1096784725
content id 11034
user id 1096784725
content id 11031
user id 385471210
content id 9538
user id 1202386221
content id 1002
user id 2018567473
content id 12148
group_num
0     275030867
0     554169193
0    1720860329
0     288641214
0    1728340777
0    1364159702
0    15216

,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,prior_question_elapsed_time,prior_question_had_explanation,prior_group_answers_correct,prior_group_responses
group_num,,,,,,,,,,
1,18,23609,275030867,5502,0,1,34000.0,False,"[0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, ...","[0, 0, 1, 1, 0, 1, 3, 3, 1, 1, 0, 3, 1, 2, 2, ..."
1,19,2035159380,1233875513,1512,0,1431,25000.0,True,NaN,NaN
1,20,2035159380,1233875513,1511,0,1431,25000.0,True,NaN,NaN
1,21,2035159380,1233875513,1513,0,1431,25000.0,True,NaN,NaN
1,22,1217231,891955351,9145,0,20,23000.0,True,NaN,NaN
1,23,11265012636,1981166446,3299,0,950,36333.0,True,NaN,NaN
1,24,11265012636,1981166446,3297,0,950,36333.0,True,NaN,NaN
1,25,11265012636,1981166446,3298,0,950,36333.0,True,NaN,NaN
1,26,4693145319,1637273633,11373,0,3148,41000.0,True,NaN,NaN


,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,prior_question_elapsed_time,prior_question_had_explanation,prior_group_answers_correct,prior_group_responses,...,user_answered_correctly_count,group,group2,bundle_id,correct_answer,part,tags,content_answered_correctly_mean,hmean_user_content_accuracy,answered_correctly
0,0,0,275030867,5729,0,0,23916.0,<NA>,[],[],...,0,9,2,5729,2,5,8,0.559251,0.613743,0
1,1,13309898705,554169193,12010,0,4427,19000.0,True,NaN,NaN,...,5199,1528339,1456244,12010,0,2,90 100 92 29,0.830091,0.764884,1
2,2,4213672059,1720860329,457,0,240,17000.0,True,NaN,NaN,...,253,9,718714,457,1,2,143 105 38 29,0.838630,0.658082,1
3,3,62798072960,288641214,13262,0,266,23000.0,True,NaN,NaN,...,273,1270332,1202390,13262,3,5,96,0.824263,0.748196,0
4,4,10585422061,1728340777,6119,0,162,72400.0,True,NaN,NaN,...,283,1955275,725759,6119,2,5,54,0.446379,0.501610,0
5,5,18020362258,1364159702,12023,0,4424,18000.0,True,NaN,NaN,...,6208,406416,372458,12023,1,2,2 107 92 29,0.544554,0.644850,1
6,6,2325432079,1521618396,574,0,1367,18000.0,True,NaN,NaN,...,2012,567958,524558,574,0,2,143 141 38 29,0.734281,0.637729,0
7,7,39456940781,1317245193,12043,0,5314,17000.0,True,NaN,NaN,...,5301,358504,327506,12043,3,2,143 114 38 29,0.829026,0.664607,1
8,8,3460555189,1700555100,7910,0,532,21000.0,True,NaN,NaN,...,744,750696,698877,7910,2,1,10 178 92,0.630756,0.684668,0
9,9,2214770464,998511398,7908,0,393,21000.0,True,NaN,NaN,...,605,1954084,1878088,7908,1,1,131 93 38,0.612414,0.571402,1


user id 275030867
content id 5502
user id 1233875513
content id 1512
user id 1233875513
content id 1511
user id 1233875513
content id 1513
user id 891955351
content id 9145
user id 1981166446
content id 3299
user id 1981166446
content id 3297
user id 1981166446
content id 3298
nothing to do
user id 1637273633
content id 11373
user id 2030979309
content id 3348
user id 2030979309
content id 3350
user id 2030979309
content id 3349
user id 319060572
content id 10951
user id 319060572
content id 10953
user id 319060572
content id 10954
user id 319060572
content id 10952
user id 288641214
content id 5418
user id 98059812
content id 5892
user id 674533997
content id 5301
user id 555691277
content id 666
user id 1317245193
content id 12195
user id 1202386221
content id 582
user id 775113212
content id 10078
user id 775113212
content id 10080
user id 775113212
content id 10081
user id 775113212
content id 10079
user id 1219481379
content id 3784
group_num
1     275030867
1    1233875513
1    1

2it [00:00,  7.01it/s]

user id 275030867
content id 6133
user id 1521618396
content id 786
nothing to do
user id 1148874033
content id 5038
group_string group2= 111101111111111111111111111111011111111111111101111111111111
group_string group2 updated= 111101111111111111111111111111011111111111111101111111111111
old value group2 = 156884
new value group2 updated= 156884
nothing to do
user id 554169193
content id 1260
user id 1281335472
content id 175
group_string group1= 1010110000000010000000000011111110000111100001111010010101100011000000010
group_string group1 updated= 1011110000000010000000000011111110000111100001111010010101100011000000010
old value group1 = 319631
new value group1 updated= 1955276
user id 998511398
content id 18
user id 2002570769
content id 6318
group_string group1= 1111101111111101111111111111111111111111111111111101001111111111111111010
group_string group1 updated= 1111101111111101111111111111111111111111111111111101101111111111111111010
old value group1 = 1055563
new value group1 upd

4it [00:00,  5.27it/s]

CPU times: user 1.88 s, sys: 86.3 ms, total: 1.97 s
Wall time: 762 ms
